### Inicializando e checando o ecossistema de databases

In [ ]:
#mongodb
!mongo --eval 'rs.initiate(); rs.isMaster().ismaster'

#redis
!redis-server --daemonize yes
!redis-cli ping

#cassandra
!cqlsh --execute="SELECT host_id, bootstrapped FROM system.local;"

### Inicializando modulos, libs e variaveis

In [ ]:
from pymongo import MongoClient
from pprintpp import pprint
import warnings
from libs.dispute_faker import generate_dispute


warnings.filterwarnings('ignore')

mongoclient = MongoClient('localhost', 27017)
dispute_db = mongoclient["disputes"]
print(dispute_db)

In [ ]:
inserted = dispute_db['disputes'].insert_many([generate_dispute() for _ in range(500)])
pprint(len(inserted.inserted_ids))

In [ ]:
! mongo --quiet --eval 'disputes.disputes.find().count();'

In [ ]:
! mongo --quiet --eval 'db.getMongo().getDBNames()'